<a href="https://colab.research.google.com/github/hitesh582/RL-CSL348-Reinforcement-Learning/blob/main/manavlawmitradataintocsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages in Colab
!pip install pdf2image pytesseract torch transformers pandas spacy tqdm ipywidgets

# Download the spaCy English language model
!python -m spacy download en_core_web_sm

# Enable Jupyter widgets extension (for interactive elements if needed)
!jupyter nbextension enable --py widgetsnbextension --sys-prefix


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json


In [ ]:
# Imports and setup
import os
from pdf2image import convert_from_path
import pytesseract
from transformers import pipeline
import pandas as pd
import spacy
from pathlib import Path
import torch
from tqdm import tqdm
from IPython.display import display, HTML

# Install dependencies for Colab
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr
!pip install pdf2image pytesseract transformers pandas spacy tqdm

# Set paths (no need to manually set in Colab since Tesseract and Poppler are installed system-wide)
POPPLER_PATH = '/usr/bin'
TESSERACT_PATH = '/usr/bin/tesseract'

# Verify installations
print(f"Checking if Poppler path exists: {os.path.exists(POPPLER_PATH)}")
print(f"Checking if Tesseract exists: {os.path.exists(TESSERACT_PATH)}")

# Check CUDA availability
if torch.cuda.is_available():
    print(f"CUDA available: True")
    print(f"Current GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("CUDA not available")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (152 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [ ]:
# Second cell - Class definition
class CourtCaseProcessor:
    def __init__(self):
        # Check if CUDA is available
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {self.device}")

        if self.device == "cuda":
            torch.cuda.set_device(0)
            print(f"Using GPU: {torch.cuda.get_device_name(0)}")

        # Initialize NLP components
        self.summarizer = pipeline("summarization",
                                   model="sshleifer/distilbart-cnn-12-6",
                                   device=0 if self.device == "cuda" else -1)
        self.nlp = spacy.load("en_core_web_sm")

    def pdf_to_text(self, pdf_path):
        """Convert PDF to text using OCR"""
        try:
            print(f"\nAttempting to convert {pdf_path} to text...")
            # No need to specify poppler_path in Colab
            images = convert_from_path(pdf_path)
            print(f"Successfully converted PDF to {len(images)} images")

            text = ""
            for i, image in enumerate(images):
                print(f"Processing page {i+1}/{len(images)}")
                page_text = pytesseract.image_to_string(image)
                text += page_text + "\n"

            print(f"Text extraction complete. Extracted {len(text)} characters")
            return text
        except Exception as e:
            print(f"Error processing {pdf_path}: {str(e)}")
            return None

    def get_sections(self, text):
        """Extract legal sections with their corresponding acts"""
        doc = self.nlp(text)
        sections = set()

        acts = {
            'IPC': 'Indian Penal Code',
            'CrPC': 'Code of Criminal Procedure',
            'CPC': 'Code of Civil Procedure',
            'IT Act': 'Information Technology Act',
            'Consumer Protection Act': 'Consumer Protection Act',
            'NDPS': 'Narcotic Drugs and Psychotropic Substances Act',
            'Prevention of Corruption Act': 'Prevention of Corruption Act'
        }

        for i, token in enumerate(doc):
            if token.text.lower() in ['section', 's.', 'sec.', 'ss.', 'sections']:
                if i + 1 < len(doc):
                    next_tokens = doc[i+1:i+10].text
                    section_num = ''
                    act_name = ''

                    for char in next_tokens.split()[0]:
                        if char.isdigit() or char in '-,':
                            section_num += char

                    for act_key in acts:
                        if act_key in next_tokens:
                            act_name = acts[act_key]
                            break

                    if section_num:
                        if act_name:
                            sections.add(f"Section {section_num} of the {act_name}")
                        else:
                            sections.add(f"Section {section_num}")

        return ', '.join(sections) if sections else 'No sections found'

    def get_summary(self, text):
        """Generate a summary focusing on case subject matter"""
        try:
            # Clean the text
            text = text.replace('\n', ' ').strip()

            # Define case subject patterns
            case_subjects = {
                'murder': ['murder', 'killed', 'death', 'homicide', '302 IPC', 'deceased'],
                'property dispute': ['property', 'land', 'possession', 'title', 'ownership', 'tenant'],
                'corruption': ['corruption', 'bribe', 'misappropriation', 'prevention of corruption act'],
                'theft': ['theft', 'stolen', 'robbery', 'burglary', '379 IPC'],
                'assault': ['assault', 'attack', 'hurt', 'injury', 'grievous', '324 IPC', '325 IPC'],
                'fraud': ['fraud', 'cheating', 'misrepresentation', 'forgery', '420 IPC'],
                'rape': ['rape', 'sexual assault', '376 IPC'],
                'dowry': ['dowry', 'dowry death', '304B IPC'],
                'drugs': ['drugs', 'NDPS', 'narcotic', 'possession of drugs'],
                'domestic violence': ['domestic violence', 'cruelty', '498A IPC'],
                'cybercrime': ['cyber', 'computer', 'online', 'IT Act'],
                'defamation': ['defamation', 'defame', '499 IPC', '500 IPC'],
                'kidnapping': ['kidnapping', 'abduction', '363 IPC'],
                'accident': ['accident', 'negligence', 'motor vehicle', 'compensation']
            }

            # Find main subject of the case
            text_lower = text.lower()
            case_subject = None
            max_matches = 0

            for subject, keywords in case_subjects.items():
                matches = sum(1 for keyword in keywords if keyword in text_lower)
                if matches > max_matches:
                    max_matches = matches
                    case_subject = subject

            # Get context around the subject
            doc = self.nlp(text)
            relevant_sentences = []

            # First, try to get facts about the case
            fact_indicators = ['facts', 'brief facts', 'prosecution case', 'allegations', 'according to']
            for sent in doc.sents:
                sent_text = sent.text.lower()
                if any(indicator in sent_text for indicator in fact_indicators):
                    if len(sent_text.split()) > 5:
                        relevant_sentences.append(sent.text)
                        if len(relevant_sentences) >= 2:
                            break

            # If no facts found, get sentences containing the case subject
            if not relevant_sentences and case_subject:
                subject_keywords = case_subjects[case_subject]
                for sent in doc.sents:
                    sent_text = sent.text.lower()
                    if any(keyword in sent_text for keyword in subject_keywords):
                        if len(sent_text.split()) > 5:
                            relevant_sentences.append(sent.text)
                            if len(relevant_sentences) >= 2:
                                break

            # Construct summary
            if case_subject:
                summary_parts = []
                summary_parts.append(f"This is a case regarding {case_subject}.")
                if relevant_sentences:
                    summary_parts.append(" ".join(relevant_sentences))
                return " ".join(summary_parts)
            else:
                return "Unable to determine the main subject of the case."

        except Exception as e:
            print(f"Error generating summary: {str(e)}")
            return "Error generating summary"

    def process_folder(self, folder_path, output_csv):
        """Process all PDFs in a folder and save results to CSV"""
        results = []
        folder = Path(folder_path)

        if not folder.exists():
            print(f"Error: Folder {folder_path} does not exist!")
            return

        pdf_files = list(folder.glob('*.pdf'))
        if not pdf_files:
            print(f"No PDF files found in {folder_path}")
            return

        print(f"Found {len(pdf_files)} PDF files to process")

        for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
            print(f"\nProcessing: {pdf_file.name}")

            text = self.pdf_to_text(str(pdf_file))
            if not text:
                print(f"Skipping {pdf_file.name} due to text extraction failure")
                continue

            try:
                sections = self.get_sections(text)
                summary = self.get_summary(text)

                result = {
                    'Summary': summary,
                    'Sections': sections,
                }
                results.append(result)

            except Exception as e:
                print(f"Error processing {pdf_file.name}: {str(e)}")

        if results:
          df=pd.DataFrame(results)
          df.to_csv(output_csv,index=False)
          print("Results saved to output_csv")
        else:
          print("No results to save to CSV")


In [ ]:
# Third cell - Run the processor
# Initialize processor
processor = CourtCaseProcessor()

# Define paths
pdf_folder = r"Dataset"  # Update this path
output_csv = "court_cases_summary.csv"

# Process the folder
processor.process_folder(pdf_folder, output_csv)

Using device: cuda
Using GPU: Tesla T4


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...


Processing: J_2023_SCC_OnLine_SC_1310_johri67_yahoocom_20240917_142228_1_2.pdf

Attempting to convert Dataset/J_2023_SCC_OnLine_SC_1310_johri67_yahoocom_20240917_142228_1_2.pdf to text...
Successfully converted PDF to 2 images
Processing page 1/2
Processing page 2/2
Text extraction complete. Extracted 5382 characters


Processing PDFs:  99%|█████████▉| 108/109 [2:14:07<01:13, 73.33s/it]


Processing: J_2001_SCC_OnLine_Bom_1125_2002_Cri_LJ_1280_johri67_yahoocom_20240927_120727_1_4.pdf

Attempting to convert Dataset/J_2001_SCC_OnLine_Bom_1125_2002_Cri_LJ_1280_johri67_yahoocom_20240927_120727_1_4.pdf to text...
Successfully converted PDF to 4 images
Processing page 1/4
Processing page 2/4
Processing page 3/4
Processing page 4/4
Text extraction complete. Extracted 12988 characters


Processing PDFs: 100%|██████████| 109/109 [2:15:03<00:00, 74.34s/it]

Results saved to output_csv
